Import libraries and set random seed

In [1]:
from gurobipy import Model, GRB, tuplelist, quicksum
import json
import random

random.seed(3)

Set scheduler parameters

In [2]:
start_time = 0
slice_size = 5*60
horizon = 24*60*60*3
print(f"Slicesize: {slice_size}s")
print(f"Scheduling Horizon: {horizon}s")

Slicesize: 300s
Scheduling Horizon: 259200s


Set request-generation parameters

In [3]:
num_requests = 500
request_min_length = 30
request_max_length = 60*60*3
priority_min = 1
priority_max = 10

Set up resources and timeslices

In [4]:
resources = ["t1" ,"t2", "t3"]
timeslices = [i for i in range(start_time, start_time + horizon, slice_size)]
print(len(timeslices))
# timeslices = [i for i in range(300)]
print(resources)
print(timeslices)

864
['t1', 't2', 't3']
[0, 300, 600, 900, 1200, 1500, 1800, 2100, 2400, 2700, 3000, 3300, 3600, 3900, 4200, 4500, 4800, 5100, 5400, 5700, 6000, 6300, 6600, 6900, 7200, 7500, 7800, 8100, 8400, 8700, 9000, 9300, 9600, 9900, 10200, 10500, 10800, 11100, 11400, 11700, 12000, 12300, 12600, 12900, 13200, 13500, 13800, 14100, 14400, 14700, 15000, 15300, 15600, 15900, 16200, 16500, 16800, 17100, 17400, 17700, 18000, 18300, 18600, 18900, 19200, 19500, 19800, 20100, 20400, 20700, 21000, 21300, 21600, 21900, 22200, 22500, 22800, 23100, 23400, 23700, 24000, 24300, 24600, 24900, 25200, 25500, 25800, 26100, 26400, 26700, 27000, 27300, 27600, 27900, 28200, 28500, 28800, 29100, 29400, 29700, 30000, 30300, 30600, 30900, 31200, 31500, 31800, 32100, 32400, 32700, 33000, 33300, 33600, 33900, 34200, 34500, 34800, 35100, 35400, 35700, 36000, 36300, 36600, 36900, 37200, 37500, 37800, 38100, 38400, 38700, 39000, 39300, 39600, 39900, 40200, 40500, 40800, 41100, 41400, 41700, 42000, 42300, 42600, 42900, 43200, 4

In [5]:
# Generate random availability for each resource
# For each resource, knock out 20% of the timeslices
resource_slices = {}
for r in resources:
    empty_hours = sorted(random.sample(timeslices, round(len(timeslices)/10)))
    resource_slices[r] = [i for i in timeslices if i not in empty_hours]
    print(r)
    print(empty_hours)
    print()

t1
[3900, 4500, 8700, 9300, 10800, 12900, 19500, 20100, 29700, 39900, 41100, 41700, 46200, 46500, 48900, 50100, 58800, 66600, 71100, 71700, 72900, 79200, 79500, 82500, 85800, 92400, 103200, 107700, 112200, 113400, 118500, 118800, 119700, 121200, 121800, 125100, 129300, 131100, 133800, 136500, 144000, 144300, 145200, 145500, 146100, 151800, 155700, 160500, 163800, 165900, 167100, 168600, 169200, 176100, 177000, 178200, 179400, 179700, 181500, 181800, 182700, 185400, 186000, 192000, 192300, 195000, 196200, 206100, 206400, 209400, 213600, 214500, 219300, 220200, 220800, 223500, 227700, 232800, 238500, 239100, 239400, 242100, 247200, 249300, 251400, 255900]

t2
[2100, 6000, 9600, 10800, 12900, 13500, 13800, 19200, 20400, 23400, 27000, 31500, 31800, 33000, 36300, 38100, 42300, 46200, 47700, 60600, 64800, 72300, 72900, 79200, 80700, 81900, 83100, 85500, 87300, 89400, 90000, 92400, 93000, 95100, 96300, 100200, 101400, 104100, 105600, 110400, 115500, 115800, 118500, 125100, 126000, 127500, 131

In [6]:
# Generate random requests
requests = {}

for i in range(num_requests):
    # NOTE: Need to come up with a method of cadencing these, as an observing window cannot exist continuously for 3 days...
    # Also it could be for a couple of hours per night, as opposed to just one long stretch.
    
    t1 = random.randint(start_time, start_time + horizon)
    t2 = random.randint(start_time, start_time + horizon)
    start = min([t1, t2])
    end = max([t1, t2])
    
    length = min(random.randint(request_min_length, request_max_length), end-start)
    
    priority = random.randint(priority_min, priority_max)
    
    requests[i] = {
            "window_start": start,
            "window_end": end,
            "length": length,
            "priority": priority
        }

for r in requests.items():
    print(r)

(0, {'window_start': 58989, 'window_end': 206309, 'length': 2011, 'priority': 1})
(1, {'window_start': 138838, 'window_end': 251024, 'length': 3156, 'priority': 6})
(2, {'window_start': 211491, 'window_end': 219206, 'length': 7715, 'priority': 3})
(3, {'window_start': 73031, 'window_end': 226421, 'length': 5602, 'priority': 2})
(4, {'window_start': 162354, 'window_end': 211355, 'length': 5687, 'priority': 10})
(5, {'window_start': 33997, 'window_end': 110434, 'length': 4813, 'priority': 9})
(6, {'window_start': 208106, 'window_end': 222815, 'length': 4472, 'priority': 8})
(7, {'window_start': 90794, 'window_end': 166246, 'length': 6860, 'priority': 5})
(8, {'window_start': 110040, 'window_end': 148984, 'length': 6739, 'priority': 1})
(9, {'window_start': 108335, 'window_end': 241333, 'length': 2585, 'priority': 4})
(10, {'window_start': 1222, 'window_end': 125135, 'length': 10231, 'priority': 9})
(11, {'window_start': 113862, 'window_end': 146504, 'length': 3668, 'priority': 1})
(12, {

### TODO: NEED TO COPY OVER THE _POSSIBLE_STARTS()_ code to more efficiently generate all the possible valid starting times for an observation

In [7]:
# Translate Requests into widx slices
slices = {}

for request_id in range(len(requests)):
    r = requests[request_id]
    window_start = r["window_start"]
    window_end = r["window_end"]
    length = r["length"]
    priority = r["priority"]

    # print(request_id, window_start, window_end, length)

    for resource, resource_slices in resource_slices.items():
        window_slices = range(window_start, window_end+1, slice_size)
        resource_window_slices = [t for t in resouce_slices if t in window_slices] # Will need to redo this part
        possible_windows = []
        for t1 in resource_window_slices:
            valid = True
            for section in range(length / slice_size):
                if (t1 + section) not in resource_window_slices:
                    valid = False
                    break

            if valid:
                possible_windows.append(t1)
                    


for rid in range(len(requests)):

    r = requests[rid]
    w_start = r["window_start"]
    w_end = r["window_end"]
    length = r["length"]
    priority = r["priority"]

    print(rid, w_start, w_end, length)
    
    for resource, hours in resource_slices.items():
        # Determine if the current start can fit in the time slices for this resource
        initial_window = range(w_start, end+1)
        # print(initial_window)
        overlap = [t for t in hours if t in initial_window]
        # print(overlap)
        possible_windows = []
        for t1 in overlap:
            valid = True
            for section in range(length):
                if (t1 + section) not in overlap:
                    valid = False
                    break
            
            if valid:
                # Add to possible_windows
                possible_windows.append(t1)
                
        print(possible_windows)
        for w in possible_windows:
            name = f"TEL-{resource}_ID-{rid}_ST-{w}"
            wnum = w + section

            slices[name] = {
                "rid": rid,
                "priority": priority,
                "resource": resource,
                "start": w,
                "duration": length
            }

    print()

for s in slices:
    print(s)
    print(slices[s])
    print()
print(aikt)

NameError: name 'resouce_slices' is not defined

In [ ]:
aikt = {}

for slice_num, slice_id in enumerate(slices):
    slice = slices[slice_id]
    for segment in range(slice["duration"]):
        win_id = slice["start"] + segment
        if win_id in aikt:
            aikt[win_id].append(slice_num)
        else:
            aikt[win_id] = [slice_num]

# for k in sorted(aikt.keys()):
    # print(k, aikt[k])

In [ ]:
m = Model("Test Schedule")

In [ ]:
requestLocations = tuplelist()
scheduled_vars = []
for r_name, r in slices.items():
    var = m.addVar(vtype=GRB.BINARY, name=str(r_name))
    scheduled_vars.append(var)
    requestLocations.append((r["rid"], r["start"], r["priority"], r["resource"], var))

m.update()

In [ ]:
# I DON'T KNOW WHY WE NEED THIS ONE
# for i, r in enumerate(slices):
    # print(r)
    # scheduled_vars[i].start = r["start"]
# m.update()

In [ ]:
# Constraint 1: One-of (only schedule one of these)
# Are we going to include these constraints in our final model? If there's no special cases, then they just turn into 
# Constraint 4 (which is why there is a skip_constraint2 flag).

In [ ]:
# # Constraint 2: And (only schedule if they can ALL be scheduled)
# # I'm going to use this to ensure that all of an observation is schedule

# NOTE: Not what this is used for. Each Request/Resource/Start has a unique IsScheduled variable, not each timeslice.

# Use this to schedule cadenced observations? And what does that mean if one of an AND condition has already been observed?
# Are the rest of the observations now fixed in place?
# Maybe this is too complicated to inject in the first run, especially as I don't  know how LCO implements these things - I don't know what observation
# requests get tagged with the AND or OR constraints (1 and 2), and how they are handled internally once part of the AND condition is fulfilled.

In [8]:
# Constraint 4: Each request only scheduled once
for rid in requests:
    match = requestLocations.select(rid, '*', '*', '*', '*', '*')
    nscheduled = quicksum([isScheduled for reqid, wnum, priority, resource, isScheduled in match])
    m.addConstr(nscheduled <= 1, f'one_per_reqid_constraint_{rid}')
m.update()

NameError: name 'requestLocations' is not defined

In [9]:
# Constraint 3: Each timeslice should only have one request in it
for r in resource_slices:
    for wnum in resource_slices[r]:
        match = requestLocations.select('*', wnum, '*', r, '*', '*')
        if len(match) == 0:
            continue
        nscheduled = quicksum(isScheduled for i, w, p, r, isScheduled in match)
        m.addConstr(nscheduled <= 1, f'one_per_slice_constraint_{r}_{wnum}')
m.update()

TypeError: 'int' object is not iterable

In [30]:
for s in aikt.keys():
    match = tuplelist()
    for slice in aikt[s]:
        match.append(requestLocations[slice])
        print(match)
    nscheduled = quicksum(isScheduled for rid, winidx, priority, resource, isScheduled in match)
    m.addConstr(nscheduled <= 1, 'one_per_slice_constraint_' + s)

<gurobi.tuplelist (1 tuples, 5 values each):
 ( 0 , 5 , 5 , t1 , <gurobi.Var TEL-t1_ID-0_ST-5> )
>
<gurobi.tuplelist (2 tuples, 5 values each):
 ( 0 , 5 , 5 , t1 , <gurobi.Var TEL-t1_ID-0_ST-5> )
 ( 1 , 5 , 4 , t1 , <gurobi.Var TEL-t1_ID-1_ST-5> )
>
<gurobi.tuplelist (3 tuples, 5 values each):
 ( 0 , 5 , 5 , t1 , <gurobi.Var TEL-t1_ID-0_ST-5> )
 ( 1 , 5 , 4 , t1 , <gurobi.Var TEL-t1_ID-1_ST-5> )
 ( 2 , 5 , 2 , t1 , <gurobi.Var TEL-t1_ID-2_ST-5> )
>
<gurobi.tuplelist (4 tuples, 5 values each):
 ( 0 , 5 , 5 , t1 , <gurobi.Var TEL-t1_ID-0_ST-5> )
 ( 1 , 5 , 4 , t1 , <gurobi.Var TEL-t1_ID-1_ST-5> )
 ( 2 , 5 , 2 , t1 , <gurobi.Var TEL-t1_ID-2_ST-5> )
 ( 4 , 5 , 2 , t1 , <gurobi.Var TEL-t1_ID-4_ST-5> )
>
<gurobi.tuplelist (5 tuples, 5 values each):
 ( 0 , 5 , 5 , t1 , <gurobi.Var TEL-t1_ID-0_ST-5> )
 ( 1 , 5 , 4 , t1 , <gurobi.Var TEL-t1_ID-1_ST-5> )
 ( 2 , 5 , 2 , t1 , <gurobi.Var TEL-t1_ID-2_ST-5> )
 ( 4 , 5 , 2 , t1 , <gurobi.Var TEL-t1_ID-4_ST-5> )
 ( 6 , 5 , 1 , t1 , <gurobi.Var TEL-t1_

In [60]:
objective = quicksum([isScheduled * (priority + 0.1/(wnum+1.0)) for req, start, priority, resource, isScheduled in requestLocations])

In [61]:
m.setObjective(objective)
m.modelSense = GRB.MAXIMIZE
m.params.MIPGap = 0.01
m.params.Method = 3
m.update()

Set parameter MIPGap to value 0.01
Set parameter Method to value 3


In [62]:
m.write("test_model.mps")

In [63]:
m.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i7-7700HQ CPU @ 2.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1198 rows, 52205 columns and 104410 nonzeros
Model fingerprint: 0xb14ab444
Variable types: 0 continuous, 52205 integer (52205 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 5e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 810.0896667
Presolve removed 127 rows and 4298 columns
Presolve time: 0.49s
Presolved: 1071 rows, 47907 columns, 95810 nonzeros
Found heuristic solution: objective 852.0896667
Variable types: 0 continuous, 47907 integer (47907 binary)
Found heuristic solution: objective 866.0903333
Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Root barrier log...

Ordering time: 0.00s

Barrier statistics:
 A

In [64]:
rxf = [isScheduled.x for rid, start, priority, resource, isScheduled in requestLocations]

In [65]:
print("ID, w, p, reso, s")
for i in range(len(requestLocations)):
    if rxf[i] == 1:
        print(requestLocations[i])

ID, w, p, reso, s
(1, 207, 4, 't1', <gurobi.Var TEL-t1_ID-1_ST-207 (value 1.0)>)
(5, 198, 5, 't1', <gurobi.Var TEL-t1_ID-5_ST-198 (value 1.0)>)
(7, 37, 4, 't3', <gurobi.Var TEL-t3_ID-7_ST-37 (value 1.0)>)
(8, 96, 2, 't1', <gurobi.Var TEL-t1_ID-8_ST-96 (value 1.0)>)
(10, 127, 2, 't3', <gurobi.Var TEL-t3_ID-10_ST-127 (value 1.0)>)
(11, 239, 3, 't1', <gurobi.Var TEL-t1_ID-11_ST-239 (value 1.0)>)
(19, 249, 3, 't3', <gurobi.Var TEL-t3_ID-19_ST-249 (value 1.0)>)
(20, 20, 4, 't3', <gurobi.Var TEL-t3_ID-20_ST-20 (value 1.0)>)
(21, 80, 2, 't1', <gurobi.Var TEL-t1_ID-21_ST-80 (value 1.0)>)
(22, 193, 4, 't3', <gurobi.Var TEL-t3_ID-22_ST-193 (value 1.0)>)
(25, 155, 5, 't3', <gurobi.Var TEL-t3_ID-25_ST-155 (value 1.0)>)
(26, 199, 5, 't2', <gurobi.Var TEL-t2_ID-26_ST-199 (value 1.0)>)
(27, 179, 4, 't3', <gurobi.Var TEL-t3_ID-27_ST-179 (value 1.0)>)
(28, 192, 4, 't2', <gurobi.Var TEL-t2_ID-28_ST-192 (value 1.0)>)
(29, 46, 4, 't3', <gurobi.Var TEL-t3_ID-29_ST-46 (value 1.0)>)
(30, 165, 1, 't3', <gurob

In [67]:
requests

{0: {'window_start': 94, 'window_end': 197, 'length': 47, 'priority': 3},
 1: {'window_start': 207, 'window_end': 233, 'length': 26, 'priority': 4},
 2: {'window_start': 51, 'window_end': 110, 'length': 41, 'priority': 3},
 3: {'window_start': 3, 'window_end': 186, 'length': 29, 'priority': 1},
 4: {'window_start': 234, 'window_end': 271, 'length': 37, 'priority': 5},
 5: {'window_start': 128, 'window_end': 253, 'length': 35, 'priority': 5},
 6: {'window_start': 98, 'window_end': 251, 'length': 20, 'priority': 1},
 7: {'window_start': 37, 'window_end': 248, 'length': 29, 'priority': 4},
 8: {'window_start': 84, 'window_end': 103, 'length': 17, 'priority': 2},
 9: {'window_start': 287, 'window_end': 295, 'length': 8, 'priority': 1},
 10: {'window_start': 91, 'window_end': 102, 'length': 11, 'priority': 2},
 11: {'window_start': 232, 'window_end': 289, 'length': 8, 'priority': 3},
 12: {'window_start': 98, 'window_end': 188, 'length': 34, 'priority': 1},
 13: {'window_start': 0, 'window_